# Finetuning models

Fine-tuning ViT models with PyTorch Lightning framework. <br/>
End2End model is intended to predict whether nodule on a given scan crop is malignant or benign. Prediction is done soley on the basis of images.<br/>
Second model, Concept model given a scan crop predict typical attributes of nodule (concepts) like: Diameter, Calcification, Subtlety and so on

# Packages

In [2]:
import torch
import numpy as np
from pytorch_lightning import LightningModule, Trainer
from pytorch_lightning.callbacks import LearningRateMonitor
from pytorch_lightning.callbacks import TQDMProgressBar

from pytorch_lightning import LightningModule, Trainer
from pytorch_lightning.callbacks import LearningRateMonitor
from pytorch_lightning.callbacks import TQDMProgressBar
from pytorch_lightning.loggers import TensorBoardLogger
from pathlib import Path

## Modules

In [3]:
%load_ext autoreload
%autoreload 2
from LIDC_Data_Module import LIDC_Data_Module
from ViT_End2End_Model_base import ViT_End2End_Model_base
from ViT_Concept_Model_base import ViT_Concept_Model_base

In [4]:
data_path = "dataset"
weights_path = "weights"
tb_logs_path = "tb_logs"

## Finetuning benign/malignant End2End ViT model

In [7]:
model_name = "ViT_end2end_32p"
Path(f"{weights_path}/End2End_ViT/{model_name}").mkdir(parents=True, exist_ok=True)
Path(f"{tb_logs_path}/End2End_ViT/{model_name}").mkdir(parents=True, exist_ok=True)

In [ ]:
# training Loop
for fold in range(5): # 5 cross validation folds
    print(f"fold: {fold}")
    trainer = Trainer(
        devices=1,
        accelerator='gpu',
        max_epochs=50,
        log_every_n_steps=22,
        logger=TensorBoardLogger(f"{tb_logs_path}/End2End_ViT/{model_name}", name=f"fold_{fold+1}"),
        enable_checkpointing=False
    )
    
    model = ViT_End2End_Model_base(learning_rate=1e-3, freeze_layers=True, train_layers=40, patch_size=32) 
    data_module = LIDC_Data_Module(
        data_dir=data_path,
        fold=fold,
        batch_size=32,
        num_workers=8,
        apply_mask=False,
        finetuning=True
    )
    trainer.fit(model, data_module)
    torch.save(model.state_dict(), f"{weights_path}/End2End_ViT/{model_name}/ViT_finetune_{fold}.pt")
    

## Finetuning ViT Concept Model

In [15]:
model_name = "ViT_concept_32p"
Path(f"{weights_path}/Concept_ViT/{model_name}").mkdir(parents=True, exist_ok=True)
Path(f"{tb_logs_path}/Concept_ViT/{model_name}").mkdir(parents=True, exist_ok=True)

In [ ]:
# training Loop
for fold in range(5): # 5 cross validation folds
    print(f"fold: {fold}")
    trainer = Trainer(
        devices=1,
        accelerator='gpu',
        max_epochs=50,
        log_every_n_steps=22,
        logger=TensorBoardLogger(f"{tb_logs_path}/Concept_ViT/{model_name}", name=f"fold_{fold+1}"),
        enable_checkpointing = False
    )
    
    model_concepts_ViT = ViT_Concept_Model_base(learning_rate=1e-3, train_layers=40, freeze_layers=True, patch_size=32)
    data_module = LIDC_Data_Module(
        data_dir=data_path,
        fold=fold,
        labels="concepts",
        batch_size=32,
        num_workers=8,
        apply_mask=False,
        finetuning=True
    )
    trainer.fit(model_concepts_ViT, data_module)
    torch.save(model_concepts_ViT.state_dict(), f"{weights_path}/Concept_ViT/{model_name}/concept_ViT_finetune_{fold}.pt")

## Evaluating End2End ViT Model

In [9]:
model_name = "ViT_end2end_32p"

In [ ]:
trainer = Trainer(devices=1,
                 accelerator="gpu"
                 )
acc = []
auroc = []
precision = []
recall = []

for fold in range(5):
    print(f"fold: {fold}")
    data_module = LIDC_Data_Module(data_dir=data_path,
                             fold=fold,
                             batch_size=32,
                             num_workers=8,
                             apply_mask=False,
                             finetuning=True
                            )
    
    model =  ViT_End2End_Model_base()
    model.load_state_dict(torch.load(f"weights/End2End_ViT/{model_name}/ViT_finetune_{fold}.pt"))
    model.eval()
    res = trainer.test(model, data_module)
    acc.append(res[0]["val_acc"])
    auroc.append(res[0]["val_auroc"])
    precision.append(res[0]["val_precision"])
    recall.append(res[0]["val_recall"])

### Average Results of the End2End ViT model:

In [11]:
np.mean(precision)

0.8720391392707825

In [12]:
np.mean(acc)

0.8875610709190369

In [13]:
np.mean(recall)

0.8986511468887329